# スクラッチを通してSVMを理解する

## 【問題1】ラグランジュの未定乗数法による最急降下

### SVMの学習は、ラグランジュの未定乗数法を用います。サンプル数分のラグランジュ乗数 $\lambda$ を用意して、以下の式により更新していきます。この計算を行うメソッドをScratchSVMClassifierクラスに実装してください。

$$
λ_i^{new}=λ_i+α(1−∑^N_{j=1} λ_jy_iy_jk(x_i,x_j))
$$

### ここで $k(x_i, x_j)$ はカーネル関数です。線形カーネルの場合は次のようになります。他のカーネル関数にも対応できるように、この部分は独立したメソッドとしておきましょう。

$$
k(x_i,x_j)=x_i^{T}x_j
$$

### 条件として、更新毎に $\lambda_i >= 0$を満たす必要があります。満たさない場合は $\lambda_i = 0$とします。

In [1]:
def _grad(self,X,y):
    
    #ラグランジュの未定乗数法による最急降下
    
    # サンプル数
    m = X.shape[0]
    
    # lamの初期化
    lam = np.random.rand(m)
    
    # lamとyを変形
    lam = lam.reshape(-1,1)
    y = y.reshape(-1,1)
    
    # lamの更新式
    Kx = _kanel(X,X.T)
    grad = np.sum(lam*(y@y.T)*Kx,axis = 0).reshape(-1,1)
    lam = lam + lr*(1 - grad)
    lam = np.where(lam < 0,0,lam)

    return 



def _kanel(self,X,y):
    # カーネル関数のメソッド
    return X @ y.T
    
    
    

## 【問題2】サポートベクターの決定

### 計算したラグランジュ乗数 $\lambda$ が設定した閾値より大きいサンプルをサポートベクターとして扱います。推定時にサポートベクターが必要になります。サポートベクターを決定し、インスタンス変数として保持しておくコードを書いてください。

### 閾値はハイパーパラメータですが、1e-5程度からはじめると良いでしょう。サポートベクターの数を出力させられるようにしておくと学習がうまく行えているかを確認できます。

In [2]:
# サポートベクターの決定
def sv_n(self,X,y):
    
    # サポートベクターの取得
    indx_sv = np.array([])
    for i in range(m):
        # 閾値よりlamが高ければ、そこに対応するXを追加
        if self.lam[i] > threshold:
            indx_sv = np.append(indx_sv,i)
            
    return indx_sv

## 【問題3】推定

### 推定時には、推定したいデータの特徴量とサポートベクターの特徴量をカーネル関数によって計算します。求めた $f(x)$ の符号が分類結果です。

$$
f(x)=∑^N_{n=1} λ_ny_{sv_n}k(x,s_n)
$$

In [3]:
def predict(self,X):
    # 予測する
    hyper = X @ self.w + self.b
    
    # 分類結果を１か−１で返す
    result = np.where(hyper > 0,1,-1)
    
    # 予測値を返却する
    return result

## 【問題4】学習と推定

## 機械学習スクラッチ入門のSprintで用意したシンプルデータセット1の2値分類に対してスクラッチ実装の学習と推定を行なってください。
## scikit-learnによる実装と比べ、正しく動いているかを確認してください。
## AccuracyやPrecision、Recallなどの指標値はscikit-learnを使用してください。

In [4]:
class ScratchSVMClassifier():
    """
    SVM分類器のスクラッチ実装
    Parameters
    ----------
    num_iter : int
      イテレーション数
    lr : float
      学習率
    kernel : str
      カーネルの種類。線形カーネル（linear）か多項式カーネル（polly）
    threshold : float
      サポートベクターを選ぶための閾値
    verbose : bool
      学習過程を出力する場合はTrue
    Attributes
    ----------
    self.n_support_vectors : int
      サポートベクターの数
    self.index_support_vectors : 次の形のndarray, shape (n_support_vectors,)
      サポートベクターのインデックス
    self.X_sv :  次の形のndarray, shape(n_support_vectors, n_features)
      サポートベクターの特徴量
    self.lam_sv :  次の形のndarray, shape(n_support_vectors, 1)
      サポートベクターの未定乗数
    self.y_sv :  次の形のndarray, shape(n_support_vectors, 1)
      サポートベクターのラベル
    """
    def __init__(self, num_iter, lr, kernel='linear',threshold=1e-5,verbose=False):
        # ハイパーパラメータを属性として記録
        self.iter = num_iter
        self.lr = lr
        self.kernel = kernel
        
        self.verbose = verbose
        
    def fit(self, X, y, X_test=None, y_test=None):
        """
        SVM分類器を学習する。検証データが入力された場合はそれに対する精度もイテレーションごとに計算する。
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        """
        # サンプル数
        self.m = X.shape[0]
        
        # 特徴量数
        self.n = X.shape[1]
               
        # 最急降下法を用いて双対問題を解く
        for _ in range(self.iter):
            self._grad(X,y)
        
        #if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            #print()
        pass
    
    def _grad(self,X,y):
    
        #ラグランジュの未定乗数法による最急降下
    
        # サンプル数
        m = X.shape[0]

        # lamの初期化
        self.lam = np.random.rand(m)

        # lamとyを変形
        self.lam = self.lam.reshape(-1,1)
        self.y = y.reshape(-1,1)

        # lamの更新式
        Kx = self._kanel(X,X)
        grad = np.sum(self.lam*(y@y.T)*Kx,axis = 0).reshape(-1,1)
        lam = self.lam + self.lr*(1 - grad)
        lam = np.where(lam < 0,0,lam)

        return lam

    def _kanel(self,X,y):
        # カーネル関数のメソッド
         
        return X @ y.T
    
    def _sv_n(self,X):
        # パラメータを０で初期化
        #
#         self.w = np.zeros(self.m)
#         self.b = 0
    
        # サポートベクターの取得
        indx_sv = np.array([])
        for i in range(125):
            
            # 閾値よりlamが高ければ、そこに対応するインデックスの番号を追加
            if self.lam[i] > 1e-5:
                indx_sv =np.append(indx_sv,i)

        # wを計算
#         for i in indx_sv:
#             self.w += self.lam[i]*y[i]*X[i]

        # bを計算
#         for i in indx_sv:
#             self.b += y[i] - (self.w @ X[i])
#         self.b /= len(indx_sv)
        
        return indx_sv


    def predict(self, X):
        """
        SVM分類器を使いラベルを推定する。
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル
        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            SVM分類器による推定結果
        """
        
        # 予測する
        indx_sv = self._sv_n(X)
        lam_y =np.array([])
        for j in range(X.shape[0]):
            for i in range(len(indx_sv)):
                ind_n = int(indx_sv[i])
                lam_yy = self.lam[ind_n]*self.y[ind_n]
                
                x_kanel = self._kanel(X[j],X[ind_n])
                
                lam_y_ = np.sum(lam_yy*x_kanel)
            lam_y = np.append(lam_y,lam_y_)

        # 分類結果を１か−１で返す
        result = np.where(lam_y > 0,-1,1)

        # 予測値を返却する
        return result


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import warnings                      #ワーニング関連のモジュール？
warnings.filterwarnings('ignore')    #ワーニングが消える？

np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, n_samples // 2)
f1 = np.random.multivariate_normal(f1, cov, n_samples // 2)
X = np.concatenate([f0, f1])
y = np.concatenate([
    np.full(n_samples // 2, 1),
    np.full(n_samples // 2, -1)
])
print(X[6])
print(X[8])
print(X[6]@X[8].T)

[-1.7604608   1.31649324]
[-2.35253112  0.5177154 ]
4.823107639160986


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
#print(y)
#print(X_train)
#print(y_train)
print(X_train.T.shape)
print(y_train.reshape(-1,1).shape)

(2, 375)
(375, 1)


In [7]:
'''SVMモデルの作成'''

#モジュール読み込み、モデル構築
svm = ScratchSVMClassifier(num_iter=1000, lr = 0.005, kernel='linear',verbose=False)

#モデルの学習

svm.fit(X_train,y_train,X_test,y_test)
y_pred = svm.predict(X_test)
print(svm._sv_n(X_test))
print(y_pred)

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124.]
[-1  1  1  1 -1  1  1 -1 -1 -1 -1  1 -1 -1  1  1  1 -1  1  1  1  1 -1 -1
  1 -1  1  1  1 -1  1  1  1 -1  1  1  1  1 -1 -1  1 -1 -1  1 -1 -1  1  1
  1  1  1  1  1  1  1 -1  1  1  1  1 -1  1  1  1  1  1  1  1  1 -1  1  1
  1  1  1  1  1  1  1  1  1  1  1 -1  1 -1  1  1  1 -1  1  1  1 -1  1 -1
  1 -1  1 -1 -1 -1 -1  1  1  1 -1 -1  1 -1 -1 -1  1 -1 -1 -1 -1  1  1  1


In [8]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


print('confusion matrix = \n', confusion_matrix(y_test, y_pred))
print('accuracy = ', accuracy_score(y_test, y_pred))
print('precision = ', precision_score(y_test, y_pred))
print('recall = ', recall_score(y_test, y_pred))
print('f1 score = ', f1_score(y_test, y_pred))


confusion matrix = 
 [[38 17]
 [ 5 65]]
accuracy =  0.824
precision =  0.7926829268292683
recall =  0.9285714285714286
f1 score =  0.855263157894737


## 【問題5】決定領域の可視化

### 決定領域を可視化してください。
### 以下の例のようにサポートベクターは異なる色で示してください。

In [9]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

def decision_region(X, y, model, step=0.01, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['versicolor', 'virginica']):
    """
    2値分類を2次元の特徴量で学習したモデルの決定領域を描く。
    背景の色が学習したモデルによる推定値から描画される。
    散布図の点は訓練データまたは検証データである。
    Parameters
    ----------------
    X : ndarray, shape(n_samples, 2)
        特徴量
    y : ndarray, shape(n_samples,)
        ラベル
    model : object
        学習したモデルのインスンタスを入れる
    step : float, (default : 0.1)
        推定値を計算する間隔を設定する
    title : str
        グラフのタイトルの文章を与える
    xlabel, ylabel : str
        軸ラベルの文章を与える
    target_names= : list of str
        凡例の一覧を与える
    """
    # setting
    scatter_color = ['red', 'blue']
    contourf_color = ['pink', 'skyblue']
    n_class = 2
    # pred
    a = np.min(X[:,0])-0.5  # Xの０列目の最小値　　-0.5
    b = np.max(X[:,0])+0.5  # Xの０列目の最大値 +0.5
    c = np.min(X[:,1])-0.5  # Xの1列目の最小値　　-0.5
    d = np.max(X[:,1])+0.5  # Xの1列目の最大値 +0.5
    ab = np.arange(a,b,step)  #(start,stop,step)の公差配列
    cd = np.arange(c,d,step)  #(start,stop,step)の公差配列
    
    mesh_f0, mesh_f1  = np.meshgrid(ab,cd)   # 引数に合わせた格子を作成（X=横、y=縦）
    
    e = np.ravel(mesh_f0)  # ravelで引数を平坦化
    f = np.ravel(mesh_f1)  # ravelで引数を平坦化
    mesh = np.c_[e,f]  # vstackと同じ、横結合  (eとfの長さが足される)
    g = model.predict(mesh)  # meshをpredict
    y_pred = g.reshape(mesh_f0.shape)
#     print(y_pred)
#     print(mesh_f0)
#     print(mesh_f1)
    # plot
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    # 等高線で分けた時の配色
    plt.contourf(mesh_f0, mesh_f1, y_pred, n_class-1, cmap=ListedColormap(contourf_color))
    # 等高線
    plt.contour(mesh_f0, mesh_f1, y_pred, n_class-1, colors='y', linewidths=3, alpha=0.5)
    for i, target in enumerate(set(y)):
        plt.scatter(X[y==target][:, 0], X[y==target][:, 1], s=80, color=scatter_color[i], label=target_names[i], marker='o')
    plt.scatter()
    patches = [mpatches.Patch(color=scatter_color[i], label=target_names[i]) for i in range(n_class)]
    plt.legend(handles=patches)
    plt.legend()
    plt.show()


In [ ]:
decision_region(X_test, y_test, svm, step=0.01, title='ScratchSVM', xlabel='xlabel', ylabel='ylabel', target_names=['0','1'])

## 扱いやすい形にする
「$\frac{M}{||w||}$ を $y_{i}(w^{T}x_{i})\geq M$ という条件の元で最大化する $w$ や $M$ を求める問題」


と表現できます。条件式は $x_{i}$ に訓練データのすべての点を入れて成り立つ必要があります。まだまだややこしいですが、これをMで割ってしまいます。そうすると、


「$\frac{1}{||w||}$ を $y_{i}(\frac{w^{T}}{M}x_{i})\geq 1$ という条件の元で最大化する $w$ や $M$ を求める問題」


になり、さらに $w^{T} \gets \frac{w^{T}}{M}$ と置き換えてしまいます。


そうすれば、


「 $\frac{1}{||w||}$ を $y_{i}(w^{T}X_{i})\geq 1$ という条件の元で最大化する $w$を求める問題」


まで簡単化できます。 $\frac{1}{||w||}$ を最大化するというのは $||w||$ を最小化することと同じです。これを後々さらに扱いやすくするために $\frac{1}{2}||w||^2$ を最小化すると考えます。よって、


「 $\frac{1}{2}||w||^2$ を $y_{i}(w^{T}x_{i})\geq 1$ という条件の元で最小化する $w$ を求める問題」


とすることができます。

## 解きやすい問題にする（双対化）
こういった不等式制約を持つ最適化問題は次のように ラグランジュの未定乗数法 で置き換えられることが知られています。


なお、このように難しい問題を別の簡単な問題に言い換えることを 双対化する といいます。


ラグランジュの未定乗数法を用いると以下のラグランジュ関数が得られます。

$$
L(w,λ)=1/2||w||^2−∑^N_{i=1} λ_i{y_i(w^Tx_i)−1}
$$

$\lambda$ はラグランジュ乗数と呼ばれる数で、0以上の値です。これを $w$について微分し、0に等しいと置くと、次の式が得られます。

$$
w=∑^N_{i=1} λ_iy_ix_i
$$

を $\lambda_{i} \geq 0$ かつ $\sum_{n=1}^{N}\lambda_{i}y_{i} = 0$ の条件の元で最大化するときの $\lambda_{i}$ を探す問題に双対化できます。


この形になれば、$\lambda$ を勾配降下法により求めることができます。$w$は出てきませんが、得られる結果は同じです。



## カーネル
最後の式の $x_{i}^{T} x_j$ の部分を $k(x_i, x_j)$ という関数に置き換えます。この関数を カーネル関数 と呼びます。

$$
L(λ) = ∑^N_{i=1} λ_i−1/2∑^N_{i=1} ∑^N_{j=1} λ_iλ_jｙ_iｙ_j(x_i,x_j)
$$

この式が問題1の最急降下法の式の元になります。


カーネル関数は $x_{i}^{T} x_j$ ではないさまざまな計算に置き換えることができます。この部分を置き換えるだけで、元の特徴量を 高次元空間 に移動させたことと同じ結果が得られ、高い分類性能を得ることができます。これを カーネルトリック と呼びます。

## 高次元へ移す簡単な例
次の図のように1次元上に2色の点があるとします。これらを直線一本を引くことで分けることは不可能です。

しかし、例えば以下のように変換してみると直線でも分けられそうです。
$x^2=2.5$ あたりに線を引くことになります。

これは$x^2$を計算し、それを縦軸にプロットしたグラフです。1次元だったデータを $\phi(x)=x^2$ の関数により高次元（2次元）へと移動しました。


こういったことをSVMはカーネルトリックにより行います。